<a href="https://colab.research.google.com/github/StetskoSergey/pida5_stetsko/blob/master/Ensemble_homework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Теперь решаем задачу регрессии - предскажем цены на недвижимость. Использовать датасет https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data (train.csv)
*Данных немного, поэтому необходимо использовать 10-fold кросс-валидацию для оценки качества моделей
Построить случайный лес, вывести важность признаков
Обучить стекинг как минимум 3х моделей, использовать хотя бы 1 линейную модель и 1 нелинейную
Для валидации модели 2-го уровня использовать отдельный hold-out датасет, как на занятии
Показать, что использование ансамблей моделей действительно улучшает качество (стекинг vs другие модели сравнивать на hold-out)
В качестве решения: Jupyter notebook с кодом, комментариями и графиками, ссылка на гитхаб*

In [0]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from jupyterthemes import jtplot
from sklearn.metrics import auc, roc_curve, roc_auc_score, mean_squared_error
from sklearn.model_selection import cross_val_score, KFold

%matplotlib inline
jtplot.style()

import warnings
warnings.filterwarnings('ignore')

In [0]:
data = pd.read_csv('train.csv')
print(data.shape)
data.head()

In [0]:
#Находим категориальные признаки
cat_feat = list(data.dtypes[data.dtypes == object].index)
print('Категориальные признаки \n',cat_feat)

#закодируем пропущенные значений строкой, факт пропущенного значения тоже может нести в себе информацию
data[cat_feat] = data[cat_feat].fillna('nan')

#отфильтруем непрерывные признаки
num_feat = [f for f in data if f not in (cat_feat + ['Id', 'SalePrice'])]
print('Непрерывные признаки \n', num_feat)

# Смотрим сколько у нас значений по каждому категориальному признаку
cat_nunique = data[cat_feat].nunique()
print(cat_nunique)

#Чтобы в разы не увеличивать число признаков при построении dummy,
#будем использовать категориальные признаки с < 30 уникальных значений
cat_feat = list(cat_nunique[cat_nunique < 30].index)

In [0]:
# Создаем дамми-переменные для категорий
dummy_train = pd.get_dummies(data[cat_feat], columns=cat_feat)

dummy_cols = list(set(dummy_train))

dummy_train = dummy_train[dummy_cols]

# Заменяем пропуски на специальное значение -999, чтобы деревья могли их отличить
X = pd.concat([data[num_feat].fillna(-999),
                     dummy_train], axis=1)

# Подготовим отдельные данные для линейной и логистической регрессии
from sklearn.preprocessing import StandardScaler

train_median = data[num_feat].median()

X_lin = pd.concat([data[num_feat].fillna(train_median),
                     data[num_feat + cat_feat].isnull().astype(np.int8).add_suffix('_NaN'),
                     dummy_train], axis=1)

scaler = StandardScaler()
scaler.fit(X_lin[num_feat])

X_lin[num_feat] = scaler.transform(X_lin[num_feat])

y = data['SalePrice']
print(X.shape)
X.head()

In [0]:
print(X_lin.shape)
X_lin.head()

### Выделим из дата сета 20% для теста

In [0]:
from sklearn.model_selection import StratifiedShuffleSplit
# Разбиваем на train/test
splitter = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=777)

for train_index, test_index in splitter.split(X, np.zeros(X.shape[0])):
    X_train = X.iloc[train_index]
    X_test = X.iloc[test_index]
    X_lin_train = X_lin.iloc[train_index]
    X_lin_test = X_lin.iloc[test_index]
    
    
    y_train = y.iloc[train_index]
    y_test = y.iloc[test_index]


### Функция для кросс валидации

In [0]:
def crossValidation(model, X, y, N = 10):
  kf = KFold(n_splits=N, shuffle=True)
  print(cross_val_score(model, X, y, cv=kf).mean())

### Обучим случайный лес И выведем важность признаков

In [0]:
from sklearn.ensemble import RandomForestClassifier

clf_rf = RandomForestClassifier(n_estimators=10, max_depth=5, min_samples_leaf=20, max_features=0.5, n_jobs=-1)
clf_rf.fit(X_train, y_train)
crossValidation(clf_rf, X_train, y_train) 

In [0]:
imp = pd.Series(clf_rf.feature_importances_)
imp.sort_values(ascending=False)

### Обучим логистическую регрессию

In [0]:
from sklearn.linear_model import LogisticRegression
clf_lr = LogisticRegression()
clf_lr.fit(X_lin_train, y_train)
crossValidation(clf_lr, X_lin_train, y_train)

In [0]:
clf_lr.predict_proba(X_lin_train)[:,3]

### Обучим линейную регрессию RidgeCV

In [0]:
from sklearn.linear_model import RidgeCV
clf_ll = RidgeCV()
clf_ll.fit(X_lin_train, y_train)
crossValidation(clf_ll, X_lin_train, y_train)


### Функции стекинга и графика ROC

In [0]:
def show_auc(y, y_pred, plot_label='', prin=True):    
    auc_val = 1 - mean_squared_error(y, y_pred, squared=False)/ np.mean(y)
    print('Процент точности предсказания по средней квадратичной ошибке в моделе {0:} : {1:.4f}'.format(plot_label,auc_val))
    

In [0]:
def get_meta_features(clf, X_train, y_train, X_test, stack_cv):
    meta_train = np.zeros_like(y_train, dtype=float)
    meta_test = np.zeros_like(y_test, dtype=float)
    
    for i, (train_ind, test_ind) in enumerate(stack_cv.split(X_train, y_train)):
        
        clf.fit(X_train.iloc[train_ind], y_train.iloc[train_ind])
        meta_train[test_ind] = clf.predict(X_train.iloc[test_ind])
        meta_test += clf.predict(X_test)
    
    return meta_train, meta_test / stack_cv.n_splits

### Стекинг

In [0]:
from sklearn.model_selection import StratifiedKFold

stack_cv = StratifiedKFold(n_splits=10, random_state=555)

meta_train = []
meta_test = []
col_names = []

print('LR features...')
meta_tr, meta_te = get_meta_features(clf_lr, X_lin_train, y_train, X_lin_test, stack_cv)

meta_train.append(meta_tr)
meta_test.append(meta_te)
col_names.append('lr_pred')

print('LL features...')
meta_tr, meta_te = get_meta_features(clf_ll, X_lin_train, y_train, X_lin_test, stack_cv)

meta_train.append(meta_tr)
meta_test.append(meta_te)
col_names.append('ll_pred')

print('RF features...')
meta_tr, meta_te = get_meta_features(clf_rf, X_train, y_train, X_test, stack_cv)

meta_train.append(meta_tr)
meta_test.append(meta_te)
col_names.append('rf_pred')

In [0]:
X_meta_train = pd.DataFrame(np.stack(meta_train, axis=1), columns=col_names)
X_meta_test = pd.DataFrame(np.stack(meta_test, axis=1), columns=col_names)

clf_lr_meta = LinearRegression()
clf_lr_meta.fit(X_meta_train, y_train)
y_pred_meta_test = clf_lr_meta.predict(X_meta_test)

show_auc(y_test, y_pred_meta_test, 'stack')


### Сравним точность предсказания Stack модели и каждой модели по отдельности - все предсказания получились менее точными

In [0]:
y_pred = clf_ll.predict(X_lin_test)
show_auc(y_test, y_pred, 'RidgeCV')

In [0]:
y_pred = clf_lr.predict(X_lin_test)
show_auc(y_test, y_pred, 'LogisticRegression')

In [0]:
y_pred = clf_rf.predict(X_test)
show_auc(y_test, y_pred, 'RandomForest')

### посмотрим вклад каждой модели в Стеке
самый большой вклад дает RidgeCV

In [0]:
pd.Series(clf_lr_meta.coef_.flatten(), index=X_meta_train.columns).plot(kind='barh')